In [2]:
import json
from madmex.wrappers import gwf_query
from madmex.models import SegmentationInformation
from importlib import import_module
from datacube.api import GridWorkflow
from madmex.util import parser_extra_args
from django.contrib.gis.geos.geometry import GEOSGeometry
from madmex.models import PredictObject

In [3]:
name_of_product = 'recipe_mex_L5_9596'
gwf_kwargs = {'region': 'Puebla', 'begin': '1995-01-01', 'end':'1996-12-31'}
gwf_kwargs.update(product=name_of_product)
iterable = gwf_query(**gwf_kwargs)
lista_iter = list(iterable)
#len(lista_iter)
[x for x in range(0,len(lista_iter)) if lista_iter[x][0] in [(40,-30), (40,-31)] ]

[24, 27]

In [52]:
lista_iter[33]

((40, -30), Tile<sources=<xarray.DataArray (time: 1)>
 array([(Dataset <id=b01c2a81-ca3d-5f0d-bae6-f39cfacca4ee type=recipe_mex_L5_9596 location=file://shared_volume/datacube/datacube_ingest/recipes/recipe_mex_L5_9596/madmex_003_40_-30_1996-01-01.nc>,)],
       dtype=object)
 Coordinates:
   * time     (time) datetime64[ns] 1996-01-01,
 	geobox=GeoBox(1667, 1667, Affine(30.0, 0.0, 2977560.0,
        0.0, -30.0, 976430.0), PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS84",6378137,6556752.3141]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",17.5],PARAMETER["standard_parallel_2",29.5],PARAMETER["latitude_of_origin",12],PARAMETER["central_meridian",-102],PARAMETER["false_easting",2500000],PARAMETER["false_northing",0]])>)

In [53]:
lista_iter[46]

((40, -31), Tile<sources=<xarray.DataArray (time: 1)>
 array([(Dataset <id=45c10b3b-8125-58c4-a461-b4647b1fcd49 type=recipe_mex_L5_9596 location=file://shared_volume/datacube/datacube_ingest/recipes/recipe_mex_L5_9596/madmex_003_40_-31_1996-01-01.nc>,)],
       dtype=object)
 Coordinates:
   * time     (time) datetime64[ns] 1996-01-01,
 	geobox=GeoBox(1667, 1667, Affine(30.0, 0.0, 2977560.0,
        0.0, -30.0, 926420.0), PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS84",6378137,6556752.3141]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["standard_parallel_1",17.5],PARAMETER["standard_parallel_2",29.5],PARAMETER["latitude_of_origin",12],PARAMETER["central_meridian",-102],PARAMETER["false_easting",2500000],PARAMETER["false_northing",0]])>)

In [20]:
import os
import re
import glob

from rasterio.io import MemoryFile
import numpy as np
import fiona
try:
    import boto3
except ImportError:
    _has_boto3 = False
else:
    _has_boto3 = True

from madmex.settings import TEMP_DIR

def list_files(bucket, path, pattern=None):
    """List objects within a 'path' in an s3 bucket
    Note that objects are listed recursively
    Args:
        bucket (str): Name of an existing s3 bucket
        path (str): Path of the folder containing the objects to be listed
        pattern (str): Optional regex like pattern to filter the returned objects
    Examples:
        >>> from madmex.util import s3
        >>> # List all files of a Landsat directory
        >>> s3.list_files(bucket='conabio-s3-oregon', path='linea_base/L8/LC080190482017022301T1-SC20180611232224')
        >>> # Filter metadata file
        >>> s3.list_files(bucket='conabio-s3-oregon', path='linea_base/L8/LC080190482017022301T1-SC20180611232224',
        ...               pattern=r'.*.xml$')
    Return:
        list: List of s3 keys (objects)
    """
    if not _has_boto3:
        raise ImportError('boto3 is required for working with s3 buckets')
    # Strip leading and trailing slash
    path = path.strip('/')
    s3 = boto3.resource('s3')
    my_bucket = s3.Bucket(bucket)
    obj_list = my_bucket.objects.filter(Prefix=path)
    out = [x.key for x in obj_list]
    if pattern is not None:
        pattern = re.compile(pattern)
        out = [x for x in out if pattern.search(x)]
    return out

In [64]:
list_files(bucket='segmentation-antares3-results', path='', pattern='.*40_-31.*shp')

['225c46_bis_seg_mex_L5_9596_landsat5_40_-31_1995_1996.shp',
 '33c811_bis_seg_mex_L5_9596_ver_comp_landsat5_40_-31_1995_1996.shp',
 '95d99e_bis_seg_mex_L5_9596_pue_comp_landsat5_40_-31_1995_1996.shp']

In [65]:
var = lista_iter[46]
bucket='segmentation-antares3-results'
filename='225c46_bis_seg_mex_L5_9596_landsat5_40_-31_1995_1996.shp'
algorithm = 'bis' 
datasource = 'landsat5'
name = 'seg_mex_L5_9596'
year = '1995_1996'
extra_args = parser_extra_args(['t=30','s=0.5','c=0.7']) 
band_list = ['ndvi_mean']
module = import_module('madmex.segmentation.%s' % algorithm)
Segmentation = module.Segmentation

In [66]:
meta, _ = SegmentationInformation.objects.get_or_create(algorithm=algorithm, datasource=datasource, parameters=json.dumps(extra_args), datasource_year=year, name=name)

In [36]:
meta.id

1

In [67]:
geoarray = GridWorkflow.load(var[1], measurements=band_list)


In [58]:
geoarray

<xarray.Dataset>
Dimensions:    (time: 1, x: 1667, y: 1667)
Coordinates:
  * time       (time) datetime64[ns] 1996-01-01
  * y          (y) float64 9.764e+05 9.764e+05 9.764e+05 ... 9.265e+05 9.264e+05
  * x          (x) float64 2.978e+06 2.978e+06 2.978e+06 ... 3.028e+06 3.028e+06
Data variables:
    ndvi_mean  (time, y, x) int16 5483 5452 5359 5325 ... 6580 6833 6883 6714
Attributes:
    crs:      PROJCS["unnamed",GEOGCS["WGS 84",DATUM["unknown",SPHEROID["WGS8...

In [68]:
seg = Segmentation.from_geoarray(geoarray, **extra_args)

In [69]:
geom = GEOSGeometry(json.dumps(seg.geobox.geographic_extent.json))

In [35]:
geom

In [70]:
shp_path = 's3://%s/%s' % (bucket, os.path.basename(filename))

In [71]:
shp_path

's3://segmentation-antares3-results/225c46_bis_seg_mex_L5_9596_landsat5_40_-31_1995_1996.shp'

In [72]:
PredictObject.objects.create(path=shp_path,
                                    the_geom=geom,
                                    segmentation_information=meta)

<PredictObject: PredictObject object (1712)>

In [39]:
pred_obj = PredictObject.objects

In [49]:
pred_obj.create(path=shp_path,the_geom=geom,segmentation_information=meta)

<PredictObject: PredictObject object (1710)>